#### Notebook to fine tune XGB Model iteratively  

**This notebook predicts TotalLOS given patient demographics, primay Diagnosis, Diagnosis category and comorbidities**  
  
The documentation from running this notebook can be found here  


### Step 1: Import all necessary libraries

In [1]:
import pandas as pd
import numpy as np
import time

import xgboost as xgb
from xgboost.sklearn import XGBRegressor

from sklearn import metrics   #Additional scklearn functions
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn import *

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 12

### Step 2: Load the data file

In [2]:
rawdata = pd.read_csv('data/DiagVsLOSRangev5.csv', index_col=[0])
data = rawdata.copy()
CatgCols = [ 'Gender', 'Ethinicity', 'Religion', 'MaritalStatus',
             'DiagnosisCategory', 'PrimaryDiag', 'DiagGroup']


### Step 3 Preprocess the data for the model 

In [3]:
### Eliminate some noise based on data exploration
# data[CatgCols[4]].value_counts()
# Make PrimaryDiag = Other if only one patient
v = data['PrimaryDiag'].value_counts() == 1 
data.loc[data['PrimaryDiag'].isin(v.index[v]), 'PrimaryDiag'] = 'Other'
# Make Religion = Other if less than 5 patients
v = data['Religion'].value_counts() < 5  
data.loc[data['Religion'].isin(v.index[v]), 'Religion'] = 'Other'
# Make DiagnosisCategory = Other if less than 10 patients
v = data['DiagnosisCategory'].value_counts() < 10  
data.loc[data['DiagnosisCategory'].isin(v.index[v]), 'DiagnosisCategory'] = 'Other'
# Make DiagGroup = Other if less than 10 patients
v = data['DiagGroup'].value_counts() < 10  
data.loc[data['DiagGroup'].isin(v.index[v]), 'DiagGroup'] = 'Other'
# Make Ethinicity = Other if less than 10 patients
v = data['Ethinicity'].value_counts() < 10  
data.loc[data['Ethinicity'].isin(v.index[v]), 'Ethinicity'] = 'Other'

### TODO  "Create a pipeline function to deal with unseen categories"

In [4]:
# one hot encoder: Mark this cell as Raw if label Encoder is in use
from sklearn.preprocessing import OneHotEncoder
# creating one hot encoder object 
data_ohe = data
for col in CatgCols:
    col_ohe = pd.get_dummies(data[col], prefix=col)
    data_ohe = pd.concat((data_ohe, col_ohe), axis=1).drop(col, axis=1)
    


In [5]:
print(data_ohe.shape)
print(data.shape)

(6287, 520)
(6287, 331)


**Choose target and unused columns in the dataset**

In [6]:
## Ignore the first three columns (LosRange, EntitySys, TotalLOS and )
target='TotalLos'
NonFeatureCols =['LosRange','EntitySys','DiagDesc','TotalLos']   # always include predictor
featureList = [x for x in data_ohe.columns if x not in NonFeatureCols]

### Step 4 Split Train and Test Data sets for validating the model

In [7]:
# Create matrices and dataframes for train/test
X = data_ohe[featureList]
y = data_ohe[target]

X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size=0.70)
X_valid, X_test,y_valid, y_test = train_test_split(X_rem, y_rem, test_size=0.5)


my_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
train_X = my_imputer.fit_transform(X_train)
test_X = my_imputer.transform(X_test)
valid_X = my_imputer.transform(X_valid)



### Import Functions for use in tuning the model

In [ ]:
import XGBFunctions as myf
#imports the following functions
#   printPredictions(alg, X, y, featureList):
#   printResults(alg, X, y, featureList):
#   modelfit(alg, xtrain, ytrain, target, featureList,useTrainCV=True, 
#            cv_folds=5, early_stopping_rounds=50):
#   printModelStats(model, dtrain, dtest, featureList, target):
#   save_tree(xgb_model, filename, rankdir='UT'):
#   show_tree(model):
#   xgbfit(alg, xtrain, ytrain, featureList):
#   xgbRfit(alg, xtrain, ytrain, featureList):

In [21]:
from xgboost import XGBRegressor
from bayes_opt import BayesianOptimization
# XGBM
# Objective function
def xgbm_re_bo(max_depth, reg_alpha, reg_lambda, learning_rate, n_estimators, subsample):
    params_gbm = {}
    params_gbm['max_depth'] = round(max_depth)
#    params_gbm['reg_alpha'] = reg_alpha
#    params_gbm['reg_lambda'] = reg_lambda
    params_gbm['learning_rate'] = learning_rate
    params_gbm['n_estimators'] = round(n_estimators)
    params_gbm['subsample'] = subsample
    scores = cross_val_score(XGBRegressor(random_state=77, **params_gbm),
                             X_train, y_train, scoring='neg_mean_absolute_error', cv=5).mean()
    score = scores.mean()
    return score
# Run Bayesian Optimization
start = time.time()
params_gbm ={
    'max_depth': (3, 10),
    'reg_alpha': (0.8, 0.9),
    'reg_lambda': (0.001, .01, 0.1),
    'learning_rate': (0.01, 0.1, 0.5, 1.0),
    'n_estimators': (80, 150, 200),
    'subsample': (0.8, 1)
}
xgbm_bo = BayesianOptimization(xgbm_re_bo, params_gbm, random_state=77)
xgbm_bo.minimize(n_iter=4)
print('It takes %s minutes' % ((time.time() - start)/60))

ValueError: setting an array element with a sequence.

ImportError: cannot import name 'TargetSpace' from 'bayes_opt' (/opt/conda/lib/python3.7/site-packages/bayes_opt/__init__.py)

In [ ]:
from sklearn.feature_selection import SelectFromModel
best_model=xgb_gs.best_estimator_
#pd.DataFrame({'Feature':SelectFromModel(best_model, max_features=20).feature_names_,
#              'Threshold':SelectFromModel(best_model, max_features=20).threshold_}, index=None)
best_model.get_params()

### Do not edit above code

**Change parameters and run to record results**

In [ ]:
%%time
xgbR1 = XGBRegressor(
    n_estimators=60,           # num_boosting_rounds passed to fit function
    use_label_encoder=False,
    enable_categorical = False,  # only for gpu_hist tree methods

    # general parameters
    max_depth= 10,              # default=6 (3-10) lower  underfits
    colsample_bytree= 1,       # cols to sample for each tree
    colsample_bynode= 1,       # cols to sample for each split node
    max_bin=5,                 ## minuimum 2  This parameter has no effect on anything. Manual Grid search.
    learning_rate = 0.1,
    verbosity= 2,              # 0:silent 1: Info 2: Warn 3: debug

    booster= 'gbtree',
    #booster parameters
    min_child_weight= 5,       ## default=1        higher underfits Train and test consistently.
    gamma=0,                   ## depends on loss function minimum loss needed to split set to regularize
    max_delta_step= 0,         # 0 is disabled, upper limit for wt neeeded to split the tree
    subsample= 1,              # sample observations for each tree 1 means all

    colsample_bylevel= 1,      # column sample at each level  Finer tuning to fix issues with data
    reg_lambda=1.0,          # L2 regularization evenly reduce of wts
    reg_alpha=0.2,             # L1 regularization eliminate weights randomly 
    # learning parameters
    objective= 'reg:squarederror',  # loss function objective (reduce squared error)  
    tree_method= 'auto',       # based on column histograms rather than reading observations every time.

    base_score= 0.5,           ## initial prediction score for all instances (global bias)
    random_state= 0,
    validate_parameters= 1,
    n_jobs=20
)
start_time = time.time()
score = cross_val_score(xgbR1, X_train, y_train, scoring='neg_mean_absolute_error', cv=5, verbose=2).mean()
#xgbR1.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=10,
#          callbacks = [xgb.callback.EarlyStopping(rounds=10, save_best=True)], verbose=True)
#print(score.mean())

elapsed = time.time() - start_time
print('Model Fit Time %s\n\n' % time.strftime("%Hh:%Mm:%Ss", time.gmtime(elapsed)))
xgbR1.get_params()

In [ ]:
# Validation Data Performance
los = y_train.values
pred = best_model.predict(X_train)
diffs = abs(los-pred).round(0).astype(int)
bins =  [-1,   7,      14,           30,          60,        90,         120,        np.inf]
names = ['a:+/-7','b:+/-14',   'c:+/-30',   'd:+/-60',   'e:+/-90', 'f:+/-120',   'g:+180']
                     
pd.cut(abs(diffs), bins, labels=names).value_counts()

In [ ]:
## Model Performance in how close predictions are to actual LOS
## Training Data Performance
los = y_train.values
pred = xgbR1.predict(X_train)
diffs = abs(los-pred).round(0).astype(int)
bins =  [-1,   7,      14,           30,          60,        90,         120,        np.inf]
names = ['a:+/-7','b:+/-14',   'c:+/-30',   'd:+/-60',   'e:+/-90', 'f:+/-120',   'g:+180']
                     
pd.cut(abs(diffs), bins, labels=names).value_counts()

In [ ]:
## Model Performance in how close predictions are to actual LOS
## Test Data Performance
los = y_test.values
pred = xgbR1.predict(X_test)
diffs = abs(los-pred).round(0).astype(int)
bins =  [-1,   7,      14,           30,          60,        90,         120,        np.inf]
names = ['a:+/-7','b:+/-14',   'c:+/-30',   'd:+/-60',   'e:+/-90', 'f:+/-120',   'g:+180']
                     
pd.cut(abs(diffs), bins, labels=names).value_counts()

In [ ]:
# Validation Data Performance
los = y_valid.values
pred = xgbR1.predict(X_valid)
diffs = abs(los-pred).round(0).astype(int)
bins =  [-1,   7,      14,           30,          60,        90,         120,        np.inf]
names = ['a:+/-7','b:+/-14',   'c:+/-30',   'd:+/-60',   'e:+/-90', 'f:+/-120',   'g:+180']
                     
pd.cut(abs(diffs), bins, labels=names).value_counts()

In [ ]:
myf.show_tree(xgbR1, tree_num=1)

In [ ]:
pd.DataFrame({'Pred':pred,'Los':los, 'Diff':pred-los}, index=None)

In [ ]:
print(dir(sklearn.feature_selection.SelectFromModel))
from sklearn.feature_selection import SelectFromModel
SelectFromModel?


In [ ]:
xgb.plot_importance(best_model, max_num_features=20)

In [ ]:
xgb.plot_importance(xgbR1, max_num_features=20)